In [22]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from mpp.models import Response
from datetime import datetime

import subprocess


In [23]:
# grab the clean text from the rds
with open('../../Response-Identification-Info/local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [37]:
response_id = 426045

response_url, response_cleaned, = session.query(
    Response.source_url,
    Response.cleaned_content
).filter(Response.id==response_id).first()

In [38]:
tfile = 'tmp/%s.xml' % response_id
with open(tfile, 'w') as f:
    f.write(response_cleaned)
    

In [39]:
script = '../../Response-Identification-Info/scripts/unique_identifier_cli.py'
cmd = 'python {0} -f {1} -u "{2}" -d True'.format(script, tfile, response_url)

In [40]:
s = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = s.communicate()

In [41]:
stdout.split('\n')

['{"original_text": "UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:[EDC:DSSDSRV]:24:BR:Browse.001:2036901365", "extraction_type": "regex", "tag": "GranuleMetaDataFile/GranuleURMetaData/BrowseProduct/BrowseGranuleId", "match_type": "urn", "potential_identifier": "UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:[EDC:DSSDSRV]:24:BR:Browse.001:2036901365"}',
 '{"original_text": "http://e4ftl01.cr.usgs.gov/MOLT/MOD11_L2.005/2000.03.12/MOD11_L2.A2000072.0520.005.2006260041825.hdf.xml", "extraction_type": "extract", "tag": "", "match_type": "url", "potential_identifier": "http://e4ftl01.cr.usgs.gov/MOLT/MOD11_L2.005/2000.03.12/MOD11_L2.A2000072.0520.005.2006260041825.hdf.xml"}',
 '']

In [42]:
stderr

''

In [36]:
cmd

'python ../../Response-Identification-Info/scripts/unique_identifier_cli.py -f tmp/426047.xml -u "http://bnb.data.bl.uk/doc/concept/lcsh/Armenia(Republic)Politicsandgovernment1991-.xml" -d True'